In [10]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors, PandasTools
!pip install mordredcommunity
from mordred import Calculator, descriptors

In [11]:
df = pd.read_csv('UnitedData.csv')
df

,String,CMC,pCMC,Temperature,Class
0,CC(CO)O,1.090000,-3.037426,23.0,Nonionic
1,C(CO)CO,1.110000,-3.045323,21.5,Nonionic
2,CCC(CO)O,0.550000,-2.740363,22.0,Nonionic
3,CC(CCO)O,0.530000,-2.724276,21.5,Nonionic
4,C(CCO)CO,0.580000,-2.763428,22.0,Nonionic
...,...,...,...,...,...
1262,CCCCCCCCCCCC(=O)NC(CO)C(=O)O,0.001620,2.790485,25.0,Nonionic
1263,CCCCCCCO[C@@H]1[C@H]([C@H]([C@@H]([C@H](O1)CO)...,0.030600,1.514279,25.0,Nonionic
1264,CCCCCCCCCCCCCC(=O)O[C@@]1([C@@H]([C@H]([C@@H](...,0.000026,4.588380,20.0,Nonionic
1265,CCCCCCCCCCOC(=O)CC(O)C[N+](C)(C)C.[Br-],0.012800,1.892790,25.0,Cationic


Добавляем дескрипторы для молекул из df с помощью библиотеки RDKit

In [12]:
df_rd = df

In [13]:
mol_list = []
for smiles in df_rd['String']:
    mol = Chem.MolFromSmiles(smiles)
    mol_list.append(mol)

In [15]:
PandasTools.AddMoleculeColumnToFrame(df_rd,'String')

In [16]:
for i,j in Descriptors.descList:
    df_rd[i] =  df_rd['ROMol'].map(j)

df_rd['Ipc'] = [Descriptors.Ipc(mol, avg=True) for mol in  df_rd['ROMol']]

In [17]:
df_rd.head()

,String,CMC,pCMC,Temperature,Class,ROMol,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CC(CO)O,1.09,-3.037426,23.0,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7fbe3fb48f90>,8.111111,8.111111,0.138889,-0.560185,...,0,0,0,0,0,0,0,0,0,0
1,C(CO)CO,1.11,-3.045323,21.5,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7fbe3fb49000>,7.906250,7.906250,0.093750,0.093750,...,0,0,0,0,0,0,0,0,0,0
2,CCC(CO)O,0.55,-2.740363,22.0,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7fbe3fb49070>,8.416667,8.416667,0.114583,-0.509259,...,0,0,0,0,0,0,0,0,0,0
3,CC(CCO)O,0.53,-2.724276,21.5,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7fbe3fb490e0>,8.392361,8.392361,0.081019,-0.351852,...,0,0,0,0,0,0,0,0,0,0
4,C(CCO)CO,0.58,-2.763428,22.0,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7fbe3fb49150>,8.086250,8.086250,0.195000,0.195000,...,0,0,0,0,0,0,0,0,1,0


In [18]:
df_rd.isna().sum()[(df_rd.isna().sum() > 0)].keys()

Index(['BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO',
       'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW'],
      dtype='object')

In [19]:
df_rd = df_rd.drop(columns=['ROMol','BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO',
       'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW'])
df_rd.shape

(1267, 214)

In [20]:
df_rd.to_csv('Data_rdkit.csv', index=False)

Отдельно создадим Dataframe, в который для молекул добавим дескрипторы из библиотеки Mordred.

In [ ]:
calc = Calculator(descriptors, ignore_3D=True,)

def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return calc(mol).asdict()
    return {}

df_mr = df['String'].apply(calculate_descriptors).apply(pd.Series)

df_mr = pd.concat([df, df_mr], axis=1)

In [ ]:
df_mr

,String,CMC,pCMC,Temperature,Class,ROMol,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,CC(CO)O,1.090000,-3.037426,23.0,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7ba8d2d44c10>,8.111111,8.111111,0.138889,-0.560185,...,6.834109,27.254130,76.052429,5.850187,18,2,16.0,14.0,3.361111,1.333333
1,C(CO)CO,1.110000,-3.045323,21.5,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7ba8d2d44c80>,7.906250,7.906250,0.093750,0.093750,...,6.192362,25.583106,76.052429,5.850187,20,2,14.0,12.0,2.75,1.500000
2,CCC(CO)O,0.550000,-2.740363,22.0,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7ba8d2d44cf0>,8.416667,8.416667,0.114583,-0.509259,...,7.280008,29.753427,90.068080,5.629255,31,4,20.0,19.0,3.611111,1.666667
3,CC(CCO)O,0.530000,-2.724276,21.5,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7ba8d2d44d60>,8.392361,8.392361,0.081019,-0.351852,...,7.131699,29.439488,90.068080,5.629255,32,3,20.0,18.0,3.611111,1.583333
4,C(CCO)CO,0.580000,-2.763428,22.0,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7ba8d2d44dd0>,8.086250,8.086250,0.195000,0.195000,...,6.608001,28.105124,90.068080,5.629255,35,3,18.0,16.0,3.0,1.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,CCCCCCCCCCCC(=O)NC(CO)C(=O)O,0.001620,2.790485,25.0,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7ba8d070f8b0>,11.438346,11.438346,0.296856,-1.200991,...,8.925188,51.014034,287.209658,5.861422,1152,21,80.0,82.0,8.333333,5.027778
1263,CCCCCCCO[C@@H]1[C@H]([C@H]([C@@H]([C@H](O1)CO)...,0.030600,1.514279,25.0,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7ba8d070f920>,9.720300,9.720300,0.423021,-1.364823,...,9.576995,51.441289,278.172939,6.181621,826,27,86.0,97.0,7.805556,4.611111
1264,CCCCCCCCCCCCCC(=O)O[C@@]1([C@@H]([C@H]([C@@H](...,0.000026,4.588380,20.0,Nonionic,<rdkit.Chem.rdchem.Mol object at 0x7ba8d070f990>,12.909030,12.909030,0.124133,-2.826419,...,10.679573,89.453878,552.314577,6.422263,5206,68,186.0,223.0,15.513889,9.034722
1265,CCCCCCCCCCOC(=O)CC(O)C[N+](C)(C)C.[Br-],0.012800,1.892790,25.0,Cationic,<rdkit.Chem.rdchem.Mol object at 0x7ba8d070fa00>,11.557246,11.557246,0.000000,-0.614754,...,9.106201,53.655147,381.187856,6.572204,2100001330,20,88.0,88.0,divide by zero encountered in power (mZagreb1),4.958333


Проверим успешность расчёта дескрипторов через эту библиотеку.

In [ ]:
df_mr.isna().sum()[(df_mr.isna().sum() > 0)].keys()

Index(['BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO',
       'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW'],
      dtype='object')

In [ ]:
df_mr.columns.value_counts()[df_mr.columns.value_counts() > 1]

,count
SlogP_VSA2,2
PEOE_VSA9,2
SlogP_VSA1,2
SMR_VSA9,2
SMR_VSA8,2
SMR_VSA7,2
SMR_VSA6,2
SMR_VSA5,2
SMR_VSA4,2
SMR_VSA3,2


Удалим дубликаты дескрипторов:

In [ ]:
# Удаление дубликатов
df_mr = df_mr.loc[:, ~df_mr.T.duplicated(keep='first')]
df_mr = df_mr.loc[:, ~df_mr.columns.duplicated(keep='first')]

In [ ]:
df_mr = df_mr.drop(columns=['ROMol','BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO',
       'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW'])

In [ ]:
df_mr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Columns: 1455 entries, String to mZagreb2
dtypes: bool(2), float64(722), int64(134), object(597)
memory usage: 14.0+ MB


Можно заметить, что значительная часть добавленных дескрипторов имеет тип данных object, что говорит о невозможности рассчитать значения для некоторых молекул. Это связано, с их строением - так как они имеют ионы в своем составе.

Из-за отсутствия данных для важной части молекул, удалим эти столбцы:

In [ ]:
df_mr.select_dtypes(include=['object']).columns

Index(['String', 'Class', 'SpAbs_A', 'SpMax_A', 'SpDiam_A', 'SpAD_A',
       'SpMAD_A', 'LogEE_A', 'VE1_A', 'VE2_A',
       ...
       'MID_C', 'AMID_C', 'MID_N', 'AMID_N', 'MID_O', 'AMID_O', 'MID_X',
       'AMID_X', 'Vabc', 'mZagreb1'],
      dtype='object', length=597)

In [ ]:
df_mr = df_mr.select_dtypes(exclude=['object'])

df_mr.insert(0, 'Class', df['Class'])
df_mr.insert(0, 'String', df['String'])

In [ ]:
df_mr.to_csv('Data_mordred.csv', index=False)

Необходимо также создать данные с "отпечатками пальцев" молекул для их характеристики, также используя библиотеку RDKit:

In [ ]:
def morgan_fpts(data):
    Morgan_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i)
        fpts =  AllChem.GetMorganFingerprintAsBitVect(mol,2,2048)
        mfpts = np.array(fpts)
        Morgan_fpts.append(mfpts)
    return np.array(Morgan_fpts)

In [ ]:
Morgan_fpts = morgan_fpts(df['String'])
Morgan_fpts.shape

[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerator
[17:01:24] DEPRECATION WARNING: please use MorganGenerat

(1267, 2048)

In [ ]:
Morgan_fp = pd.DataFrame(Morgan_fpts,columns=['Col_{}'.format(i) for i in range(Morgan_fpts.shape[1])])
Morgan_fp = pd.concat([df.iloc[:, :5], Morgan_fp], axis=1)

In [ ]:
Morgan_fp

,String,CMC,pCMC,Temperature,Class,Col_0,Col_1,Col_2,Col_3,Col_4,...,Col_2038,Col_2039,Col_2040,Col_2041,Col_2042,Col_2043,Col_2044,Col_2045,Col_2046,Col_2047
0,CC(CO)O,1.090000,-3.037426,23.0,Nonionic,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,C(CO)CO,1.110000,-3.045323,21.5,Nonionic,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CCC(CO)O,0.550000,-2.740363,22.0,Nonionic,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,CC(CCO)O,0.530000,-2.724276,21.5,Nonionic,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,C(CCO)CO,0.580000,-2.763428,22.0,Nonionic,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,CCCCCCCCCCCC(=O)NC(CO)C(=O)O,0.001620,2.790485,25.0,Nonionic,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1263,CCCCCCCO[C@@H]1[C@H]([C@H]([C@@H]([C@H](O1)CO)...,0.030600,1.514279,25.0,Nonionic,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1264,CCCCCCCCCCCCCC(=O)O[C@@]1([C@@H]([C@H]([C@@H](...,0.000026,4.588380,20.0,Nonionic,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1265,CCCCCCCCCCOC(=O)CC(O)C[N+](C)(C)C.[Br-],0.012800,1.892790,25.0,Cationic,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
Morgan_fp.info()
Morgan_fp.to_csv('Data_fps.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Columns: 2053 entries, String to Col_2047
dtypes: float64(3), int64(2048), object(2)
memory usage: 19.8+ MB


Сохраним таблицу с дескрипторами RDKit в sqlite:

In [25]:
import sqlite3

conn = sqlite3.connect('df_rd.db')

df_rd.to_sql('my_table', conn, if_exists='replace', index=False)

conn.close()